# Разработка LL(1)-анализатора



In [22]:
grammar = {
        "PROGRAM_START": [["PROGRAM_CORE", "PROGRAM_CORE_NEXT"]], # начало программы
        # основная часть программы
        "PROGRAM_CORE": [
            ["DEF_OP"], # определение переменной
            ["EXPRESSION", ";"], # выражение
            ["OPERATOR_IF"], # условие
            ["OPERATOR_FOR"] # цикл
        ],
        # следующая часть программы
        "PROGRAM_CORE_NEXT":[
            ["PROGRAM_CORE", "PROGRAM_CORE_NEXT"],
            ["ϵ"] # пустота
        ],

        # определение переменной
        "DEF_OP": [["DEF_OP_TYPE_ID", "DEF_OP_NEXT"]],
        # определение типа и идентификатора
        "DEF_OP_TYPE_ID": [["TYPE", "identifier"]],
        "TYPE": [
            ["int"],
            ["bool"],
            ["float"],
            ["string"]
        ],
        # следующая часть определения
        "DEF_OP_NEXT": [
            ["DEF_OP_FUNC_NEXT"],
            ["DEF_OP_VAR_NEXT", ";"]
        ],

        # определение переменной
        "DEF_OP_VAR": [["DEF_OP_TYPE_ID", "DEF_OP_VAR_NEXT"]],
        "DEF_OP_VAR_NEXT": [
            ["=", "EXPRESSION"],
            ["ϵ"]
        ],
        # определения функции
        "DEF_OP_FUNC": [["DEF_OP_TYPE_ID", "DEF_OP_FUNC_NEXT"]],
        "DEF_OP_FUNC_NEXT": [["(", "DEF_OP_FUNC_PARAMS", ")", "{", "BODY", "}"]],
        "DEF_OP_FUNC_PARAMS": [
            ["DEF_OP_TYPE_ID", "DEF_OP_FUNC_PARAMETERS_LIST"],
            ["ϵ"]
        ],
        # определения списка параметров функции
        "DEF_OP_FUNC_PARAMETERS_LIST": [
            [",", "DEF_OP_TYPE_ID", "DEF_OP_FUNC_PARAMETERS_LIST"],
            ["ϵ"]
        ],
        # определения вызовов функции
        "CALL_OP":[["identifier", "CALL_OP_NEXT"]],
        "CALL_OP_NEXT": [
            ["CALL_OP_FUNC_NEXT"],
            ["ϵ"]
        ],
        # определения идентификатора вызываемой функции
        "CALL_OP_FUNC": [["identifier", "CALL_OP_FUNC_NEXT"]],
        "CALL_OP_FUNC_NEXT": [["(", "CALL_OP_FUNC_PARAMETERS", ")"]],
        "CALL_OP_FUNC_PARAMETERS": [
            ["identifier", "CALL_OP_FUNC_PARAMETERS_LIST"],
            ["ϵ"]
        ],
        "CALL_OP_FUNC_PARAMETERS_LIST": [
            [",", "identifier", "CALL_OP_FUNC_PARAMETERS"],
            ["ϵ"]
        ],
        # тело блока
        "BODY": [["BODY_FIRST", "BODY_NEXT"]],
        "BODY_FIRST": [
            ["DEF_OP_VAR", ";"],
            ["EXPRESSION", ";"],
            ["OPERATOR_IF"],
            ["OPERATOR_FOR"],
            ["OPERATOR_WHILE"],
            [";"]
        ],
        "BODY_NEXT": [
            ["BODY_FIRST", "BODY_NEXT"],
            ["ϵ"]
        ],
        # операторы
        "OPERATOR_IF": [["if", "(", "EXPRESSION", ")", "{", "BODY", "}"]],
        "OPERATOR_FOR": [["for", "(", "DEF_OP_VAR", ";", "EXPRESSION", ";", "EXPRESSION", ")", "{", "BODY", "}"]],
        "OPERATOR_WHILE": [["while", "(", "EXPRESSION", ")", "{", "BODY", "}"]],
        # выражение
        "EXPRESSION": [["L_OR", "L_OR_LIST"]],
        # логическое ИЛИ
        "L_OR_LIST": [
            ["or", "L_OR", "L_OR_LIST"],
            ["ϵ"]
        ],
        "L_OR": [["L_AND", "L_AND_LIST"]],
        # логическое И
        "L_AND_LIST": [
            ["and", "L_AND", "L_AND_LIST"],
            ["ϵ"]
        ],
        "L_AND": [["L_CMP", "L_CMP_LIST"]],
        "L_CMP_LIST": [
            ["==", "L_CMP", "L_CMP_LIST"],
            ["!=", "L_CMP", "L_CMP_LIST"],
            ["<", "L_CMP", "L_CMP_LIST"],
            [">", "L_CMP", "L_CMP_LIST"],
            ["ϵ"]
        ],
        "L_CMP": [["A", "A_LIST"]],
        "A_LIST": [
            ["+", "A", "A_LIST"],
            ["-", "A", "A_LIST"],
            ["ϵ"]
        ],
        # арифметика
        "A": [["M", "M_LIST"]],
        "M_LIST": [
            ["*", "M", "M_LIST"],
            ["/", "M", "M_LIST"],
            ["ϵ"]
        ],
        # терминалы
        "M": [
            ["const"],
            ["true"],
            ["false"],
            ["CALL_OP"],
            ["not", "M"],
            ["(", "EXPRESSION", ")", "M_AFTER_BRACKETS"]
        ],
        "M_AFTER_BRACKETS": [
            ["EXPRESSION"],
            ["ϵ"]
        ]
    }


In [23]:
import pandas as pd
import re

In [24]:
class CFG_rule: # правило
  def __init__(self, symbols):
     self.left_part_symbol = symbols[0]
     self.right_part_symbols = [symbol for symbol in symbols[1:]]
     self.rule_text = self.left_part_symbol + " ::= " + " ".join(self.right_part_symbols)
     self.non_terminal_symbols = set()
     self.terminal_symbols = set()
     self._fill_symbols_type_lists()


  def _fill_symbols_type_lists(self):
    for symbol in self.right_part_symbols:
      if symbol.isupper(): # если верхний регистр, то правило уходит в нетерминальные символы
        self.non_terminal_symbols.add(symbol)
      else: # иначе в терминальные
        self.terminal_symbols.add(symbol)
    self.non_terminal_symbols.add(self.left_part_symbol)

In [25]:
class CFG:
  def __init__(self):
    self.rules = []
    self.non_terminal_symbols = set()
    self.terminal_symbols = set()
    self.first = {}
    self.follow = {}
    self.parsing_table = None


  def add_new_rules(self, *CFG_rules):
    for CFG_rule in CFG_rules:
      self.rules.append(CFG_rule) # добавляем правила в список плавил
      self.non_terminal_symbols = self.non_terminal_symbols.union(CFG_rule.non_terminal_symbols) # объединяем нетерминальные символы
      self.terminal_symbols = self.terminal_symbols.union(CFG_rule.terminal_symbols) # объединяем терминальные символы


  def print_rules(self):
    for rule in self.rules:
      print(rule.rule_text)


  def fill_first_set(self):
    for rule in self.rules:
      self.first[rule.left_part_symbol] = set()

    changed = True
    while changed:
      changed = False
      for rule in self.rules:
        set_len = len(self.first[rule.left_part_symbol])
        if rule.right_part_symbols[0] in rule.terminal_symbols:
          self.first[rule.left_part_symbol].update([rule.right_part_symbols[0]])
        elif rule.right_part_symbols[0] in rule.non_terminal_symbols:
          self.first[rule.left_part_symbol].update(self.first[rule.right_part_symbols[0]])
        if set_len - len(self.first[rule.left_part_symbol]) != 0:
          changed = True


  def fill_follow_set(self):
    for non_terminal in self.non_terminal_symbols:
      self.follow[non_terminal] = set()
    self.follow["PROGRAM_START"].update("$")
    changed = True
    while changed:
      changed = False
      for rule in self.rules:
        for index, right_symbol in enumerate(rule.right_part_symbols):
            if right_symbol not in self.non_terminal_symbols:
              continue
            set_len = len(self.follow[right_symbol])
            if len(rule.right_part_symbols) != index + 1:
              for s in rule.right_part_symbols[index + 1:]:
                if s not in self.non_terminal_symbols:
                  self.follow[right_symbol].update(s)
                  continue
                self.follow[right_symbol] = self.follow[right_symbol].union(self.first[s]).difference("ϵ")
                if "ϵ" in self.first[s]:
                  self.follow[right_symbol] = self.follow[right_symbol].union(self.follow[rule.left_part_symbol])
            elif len(rule.right_part_symbols) == index + 1:
                  self.follow[right_symbol] = self.follow[right_symbol].union(self.follow[rule.left_part_symbol])
            if set_len - len(self.follow[right_symbol]) != 0:
              changed = True


  def fill_parsing_table(self):
    self.parsing_table = pd.DataFrame(data = "error", index = list(self.non_terminal_symbols), columns=list(self.terminal_symbols))
    self.parsing_table["$"] = "error"
    self.parsing_table = self.parsing_table.drop(columns=["ϵ"])

    for rule in self.rules:
      first_set = set()
      flag = True
      for symbol in rule.right_part_symbols:
        if symbol in rule.terminal_symbols:
          first_set.update([symbol])
          flag = False
          break
        first_set = first_set.union(self.first[symbol]).difference("ϵ")
        if "ϵ" not in self.first[symbol]:
          flag = False
          break
      if flag:
        first_set.update("ϵ")
      for terminal_symbol in first_set.difference("ϵ"):
        self.parsing_table.loc[rule.left_part_symbol][terminal_symbol] = rule
      if "ϵ" in first_set:
        for term in self.follow[rule.left_part_symbol]:
          self.parsing_table.loc[rule.left_part_symbol][term] = rule
      if "ϵ" in first_set and "$" in self.follow[rule.left_part_symbol]:
        self.parsing_table.loc[rule.left_part_symbol]["$"] = rule


  def update_sets(self):
    self.fill_first_set()
    self.fill_follow_set()
    self.fill_parsing_table()

In [26]:
def read_rule_from_grammar(left_symbol, right_parts): # чтение правил из грамматики
  res = [] # будущий список правил
  for symbol in right_parts:
    if isinstance(symbol, list): # если текущий символ - list
      res.append([left_symbol] + read_rule_from_grammar(left_symbol, symbol)) # то ещё раз (рекурс.) вызываем функцию
    else:
      res.append(symbol)
  return res

In [33]:
CFG = CFG()
for key, value in grammar.items():
  rules = read_rule_from_grammar(key, value)
  for rule in rules:
    CFG.add_new_rules(
        CFG_rule(rule)
        )
CFG.update_sets()
print(f"Множество first: {CFG.first}")
print("-------")
print(f"Множество follow: {CFG.follow}")

Множество first: {'PROGRAM_START': {'not', 'if', 'identifier', 'float', 'const', 'string', 'true', 'bool', 'for', 'int', 'false', '('}, 'PROGRAM_CORE': {'string', 'true', 'not', 'if', 'identifier', 'bool', 'for', 'float', 'int', 'const', 'false', '('}, 'PROGRAM_CORE_NEXT': {'ϵ', 'not', 'if', 'float', 'identifier', 'const', 'string', 'true', 'bool', 'for', 'int', 'false', '('}, 'DEF_OP': {'string', 'bool', 'float', 'int'}, 'DEF_OP_TYPE_ID': {'string', 'bool', 'float', 'int'}, 'TYPE': {'float', 'int', 'string', 'bool'}, 'DEF_OP_NEXT': {'ϵ', '=', '('}, 'DEF_OP_VAR': {'string', 'bool', 'float', 'int'}, 'DEF_OP_VAR_NEXT': {'ϵ', '='}, 'DEF_OP_FUNC': {'string', 'bool', 'float', 'int'}, 'DEF_OP_FUNC_NEXT': {'('}, 'DEF_OP_FUNC_PARAMS': {'ϵ', 'string', 'bool', 'float', 'int'}, 'DEF_OP_FUNC_PARAMETERS_LIST': {'ϵ', ','}, 'CALL_OP': {'identifier'}, 'CALL_OP_NEXT': {'ϵ', '('}, 'CALL_OP_FUNC': {'identifier'}, 'CALL_OP_FUNC_NEXT': {'('}, 'CALL_OP_FUNC_PARAMETERS': {'identifier', 'ϵ'}, 'CALL_OP_FUNC_PA

In [35]:
lexem_types = {
      "reserved_words" : ["if", "else", "elif", "int", "id", "for", "while", "bool", "string", "and", "or", "not"],
      "operator" : ["+", "-", "<", ">", "==", "=", "!=", ">=", "<=", "*", "/"],
      "separator" : [";", ",", "{", "}", "(", ")"],
  }

In [29]:
class Lexical_analyzer:
  def __init__(self, lexem_types, terminal_symbols):
    self.lexem_types = lexem_types
    self.terminal_symbols = terminal_symbols


  def get_lexems(self, input_string):
    current_word = ""
    lexems = []
    for symbol in input_string:
      if symbol in self.lexem_types["reserved_words"] or symbol in self.lexem_types["operator"] \
      or symbol in self.lexem_types["separator"] or symbol == " ":
        if current_word != "":
          lexems.append(current_word)
        if symbol != " ":
          lexems.append(symbol)
        current_word = ""
      else:
        current_word += symbol
    return lexems


  def classify_lexemes(self, lexems):
    classified_lexems = {}
    for lexem in lexems:
      if lexem in self.terminal_symbols:
        classified_lexems[lexem] = lexem
      elif re.match(r"^\d+$", lexem):
        classified_lexems[lexem] = "int"
      elif re.match(r"^\w+$", lexem):
        classified_lexems[lexem] = "identifier"
    return classified_lexems


  def get_lexems_to_ll_parser(self, classified_lexems):
    return [lexem_type for lexem_type in classified_lexems.values()]

In [39]:
class LL1_Parser:
  def __init__(self, CFG):
    self.current_CFG = CFG

  def check_input_text(self, input_words, logging = False):
    stack = [f"PROGRAM_START", "$"]
    stack_symbol = stack[0]
    symbol_pointer = 0
    while stack_symbol != "$":
      current_symbol = input_words[symbol_pointer]
      if logging:
        print(f"Стек: {stack}\nТекущий символ стека: {stack_symbol}\nТекущий символ входной строки: {current_symbol}")
        print(f"Входная строка {input_words[symbol_pointer:]}")
      if stack_symbol == current_symbol:
        stack.remove(current_symbol)
        symbol_pointer += 1
      elif stack_symbol in self.current_CFG.terminal_symbols:
        return False
      elif self.current_CFG.parsing_table.loc[stack_symbol][current_symbol] == "error":
        return False
      elif isinstance(self.current_CFG.parsing_table.loc[stack_symbol][current_symbol], CFG_rule):
        stack.remove(stack_symbol)
        if logging:
          print(f'- ПРАВИЛО: {self.current_CFG.parsing_table.loc[stack_symbol][current_symbol].rule_text}')
        for symbol in self.current_CFG.parsing_table.loc[stack_symbol][current_symbol].right_part_symbols[::-1]:
          if symbol != "ϵ":
            stack.insert(0, symbol)
      stack_symbol = stack[0]
      if logging:
        print("----------------")
    return True


In [40]:
LA = Lexical_analyzer(lexem_types, CFG.terminal_symbols)
words_to_ll_parser_list = [
    ['int', 'identifier', ';', '$'],
    ['for', '(', 'int', 'identifier', 'identifier', "identifier", 'identifier', 'identifier', '<', 'int', 'int', '+', '+', '==', ')', '{', 'identifier', 'int', '}', '$'],
    ]

In [41]:
LL1 = LL1_Parser(CFG)
for test_words in words_to_ll_parser_list:
  if LL1.check_input_text(test_words, True):
    print(f'{test_words} соответствует грамматике')
    print("----------------")
  else:
    print(f'{test_words} не соответствует грамматике')
    print("----------------")

Стек: ['PROGRAM_START', '$']
Текущий символ стека: PROGRAM_START
Текущий символ входной строки: int
Входная строка ['int', 'identifier', ';', '$']
- ПРАВИЛО: PROGRAM_START ::= PROGRAM_CORE PROGRAM_CORE_NEXT
----------------
Стек: ['PROGRAM_CORE', 'PROGRAM_CORE_NEXT', '$']
Текущий символ стека: PROGRAM_CORE
Текущий символ входной строки: int
Входная строка ['int', 'identifier', ';', '$']
- ПРАВИЛО: PROGRAM_CORE ::= DEF_OP
----------------
Стек: ['DEF_OP', 'PROGRAM_CORE_NEXT', '$']
Текущий символ стека: DEF_OP
Текущий символ входной строки: int
Входная строка ['int', 'identifier', ';', '$']
- ПРАВИЛО: DEF_OP ::= DEF_OP_TYPE_ID DEF_OP_NEXT
----------------
Стек: ['DEF_OP_TYPE_ID', 'DEF_OP_NEXT', 'PROGRAM_CORE_NEXT', '$']
Текущий символ стека: DEF_OP_TYPE_ID
Текущий символ входной строки: int
Входная строка ['int', 'identifier', ';', '$']
- ПРАВИЛО: DEF_OP_TYPE_ID ::= TYPE identifier
----------------
Стек: ['TYPE', 'identifier', 'DEF_OP_NEXT', 'PROGRAM_CORE_NEXT', '$']
Текущий символ стека